### Betterway 37. 내장 타입을 여러 단계로 내포시기보다는 클래스를 합성하라

- 확장성을 고려 하지 않은 클래스의 생성으로 내부 에서 변경이 필요할 때 내부 코드가 깨질 확률이 크다. 

In [8]:
class SimpleGradeBook :
    def __init__(self) :
        self.grades = {}

    def add_student(self, name) :
        self.grades[name] = []
    
    def report_grade(self, name, score) :
        self.grades[name].append(score)

    def average_grade(self, name) :
        grade = self.grades[name]
        return sum(grade) / len(grade)


In [12]:
book = SimpleGradeBook()

book.add_student("이주환")
book.report_grade("이주환", 90)
book.report_grade("이주환", 95)
book.report_grade("이주환", 85)

print(book.grades)
print(book.average_grade("이주환"))


90.0
{'이주환': [90, 95, 85]}


- 만약, 과목 별 성적을 리스트로 저장 하고 싶다고 할 때 내부 코드는 이렇게 변경 할 수 있다.

In [44]:
# defaultdict를 사용 한 값이 없는 딕셔너리에 대응 하는 방법 유지
from collections import defaultdict

class BySubjectGradeBook :
    def __init__(self) :
        self.grades = {}

    def add_student(self, name) :
        self.grades[name] = defaultdict(list)

    def report_grade(self, name, subject, grade) :
        by_subject = self.grades[name]
        # print(f"by_subject : {by_subject}")
        grade_list = by_subject[subject]
        # print(f"grade_list : {grade_list}")
        grade_list.append(grade)
        # print(f"grade_list after append : {grade_list}")

    def average_grade(self, name) :
        by_subject = self.grades[name]
        total, count = 0, 0
        for val in by_subject.values() :
            total += sum(val)
            count += len(val)
        print(by_subject)
        return total / count


- 아직 까지 코드의 복잡도는 관리할 수 있는 정도이다.

In [45]:
book = BySubjectGradeBook() 

book.add_student("이주환")
book.report_grade("이주환", "수학", 75)
book.report_grade("이주환", "수학", 65)
book.report_grade("이주환", "체육", 90)
book.report_grade("이주환", "체육", 90)

print(f"평균 : {book.average_grade('이주환')}")

defaultdict(<class 'list'>, {'수학': [75, 65], '체육': [90, 90]})
평균 : 80.0


- 하지만, 이렇게 설계 된 코드에서 요구사항이 또 바뀌게 되어 내부 코드를 수정 해야한다.

    - 과목의 점수에 가중치를 저장 하여 기말고사와 중간고사가 일반 쪽지시험 보다 성적에 더 영향이 미치게 코드를 변경 하는 것이다.

    - 코드를 수정하니, report_grade는 튜플로 저장하는 방식만 변경 되었을 뿐 차이가 별로 없다. 하지만, 평균을 보여주는 메서드는 튜플을 언패킹 해야 하기 때문에 복잡하게 관리 되면서 가독성이 떨어진다.    

In [61]:
class WeightedGradeBook :
    def __init__(self) :
        self.grades = {}

    def add_student(self, name) :
        self.grades[name] = defaultdict(list)

    def report_grade(self, name, subject, score, weight) :
        by_subject = self.grades[name]
        grade_list = by_subject[subject]
        grade_list.append((score, weight))

    def average_grade(self, name) :
        by_subject = self.grades[name]
        score_sum, score_count = 0, 0 
        
        for subject, scores in by_subject.items() :
            subject_avg, total_weight = 0, 0
            print(f"subject : {subject}")
            print(f"score : {scores}")
            for score, weight in scores :
                print("##### unpack for loop ######")
                print(f"score : {score}")
                print(f"weight : {weight}")
                subject_avg += score * weight
                total_weight += weight
                print(f"과목 평균 : {subject_avg}")
                print(f"총 가중치 : {total_weight}")
            
            score_sum += subject_avg / total_weight
            score_count += 1
            print(f"score_sum : {score_sum}")
            print(f"score counting : {score_count}")
        return score_sum / score_count

In [62]:
book = WeightedGradeBook() 

book.add_student("이주환")
book.report_grade("이주환", "수학", 75, 0.05)
book.report_grade("이주환", "수학", 65, 0.015)
book.report_grade("이주환", "수학", 70, 0.80)
book.report_grade("이주환", "체육", 100, 0.40)
book.report_grade("이주환", "체육", 85, 0.60)

print(f" 평균 : {book.average_grade('이주환'):.2f}%")

subject : 수학
score : [(75, 0.05), (65, 0.015), (70, 0.8)]
##### unpack for loop ######
score : 75
weight : 0.05
과목 평균 : 3.75
총 가중치 : 0.05
##### unpack for loop ######
score : 65
weight : 0.015
과목 평균 : 4.725
총 가중치 : 0.065
##### unpack for loop ######
score : 70
weight : 0.8
과목 평균 : 60.725
총 가중치 : 0.865
score_sum : 70.20231213872833
score counting : 1
subject : 체육
score : [(100, 0.4), (85, 0.6)]
##### unpack for loop ######
score : 100
weight : 0.4
과목 평균 : 40.0
총 가중치 : 0.4
##### unpack for loop ######
score : 85
weight : 0.6
과목 평균 : 91.0
총 가중치 : 1.0
score_sum : 161.20231213872833
score counting : 2
 평균 : 80.60%


In [65]:
grades = []
grades.append((95, 0.45))
grades.append((85, 0.55))

total = sum(score * weight for score, weight in grades)
total_weight = sum(weight for _, weight in grades)
average_grade = total / total_weight

print(average_grade)

89.5


In [66]:
grades = []
grades.append((95, 0.45,"참 잘했어요"))
grades.append((85, 0.55,"조금만 더 열심히"))

total = sum(score * weight for score, weight,_ in grades)
total_weight = sum(weight for _, weight,_ in grades)

average_grade = total / total_weight

print(average_grade)

89.5


- nametuple을 이용 하여 인덱스에 의존 하는 코드를 작성 하지 않고, 정해진 이름을 호출 할 수 있다. 

    > namedtuple의 한계
    1. 클래스에는 디폴트 인자 값을 지정할 수 없다. 그렇기 때문에 속성이 4 - 5 개 보다 더 많은 경우는 dataclasses 내장 모듈을 사용 하는 편이 낫다고 함
    
    2. namedtuple을 사용 하더라도 인스턴스의 어트리뷰트 값을 숫자 인덱스를 통해 접근 할 수 있다. 그렇기 때문에 namedtuple을 온전히 제어할 수 있는 상황이 아니라면 새로운 클래스를 정의 하는 편이 낫다고 안내하고 있음

    

In [113]:
from collections import namedtuple

Grade = namedtuple("Grade", ("score", "weight"))

class Subject :
    def __init__(self) :
        self.grades = []

    def report_grade(self, score, weight) :
        self.grades.append(Grade(score, weight))

    def average_grade(self) :
        total, total_weight = 0, 0
        for grade in self.grades :
            # print(f"score : {grade.score}, weight : {grade.weight}")
            total += grade.score * grade.weight
            total_weight += grade.weight
        return total / total_weight

class Student :
    def __init__(self) :
        self.subjects = defaultdict(Subject)
        
    def get_subject(self, name) :
        return self.subjects[name]
    
    def average_grade(self) :
        total, count = 0, 0
        for subject in self.subjects.values() :
            total += subject.average_grade()
            count += 1
        return total / count

class GradeBook :
    def __init__(self) :
        self.students = defaultdict(Student)
        
    def get_students(self, name) :
        return self.students[name]


In [114]:
book = GradeBook()

juhwan = book.get_students("이주환")
math = juhwan.get_subject("주환")
math.report_grade(75, 0.05)
math.report_grade(65, 0.15)
math.report_grade(70, 0.80)

gym = juhwan.get_subject("체육")
gym.report_grade(100, 0.40)
gym.report_grade(85, 0.60)

print(juhwan.average_grade())

80.25
